In [ ]:
from flask import Flask,request,redirect,render_template,url_for
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import pandas as pd
app=Flask(__name__)
@app.route('/',methods=['POST','GET'])
def save():
    if request.method == 'POST':
        #Load Model
        datas1=pd.read_csv('newdata.csv')
        #training set
        from sklearn.model_selection import train_test_split
        x=datas1["Description"]
        y=datas1["output"]
        x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
        print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
        
        from sklearn.feature_extraction.text import TfidfVectorizer#each vector calculate the 
        vectorizer = TfidfVectorizer( max_df= 0.9).fit(x_train)
        X_train = vectorizer.transform(x_train)
        X_test = vectorizer.transform(x_test)
        
        from sklearn.linear_model import LogisticRegression
        model = LogisticRegression(C=.1, class_weight='balanced')
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        from sklearn.metrics import accuracy_score
        print("Training Accuracy : ", accuracy_score(y_train, y_pred_train))
        print("Testing Accuracy  : ", accuracy_score(y_test, y_pred_test))
        
        #model = pickle.load(open('review.h5','rb'))
        text = request.form['ReviewText']
        #print(text)
        tfidf = vectorizer.transform([text])
        preds = model.predict_proba(tfidf)
        preds=list(preds)
        a=preds.index(max(preds))
        if a== 0:
            return render_template('nutral.html')
        elif a==-1:
            return render_template('negative.html')
        elif a==1:
            return render_template('positive.html')
    return render_template('review.html')

if __name__ == '__main__':
    app.run(debug=False)
        
    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
